# 1.3 By Classical Shadow
## Multiple Experiments

Consider a scenario, you have multiple circuits that you want to run at once.

Call `.measure()` one by one will be inefficient,
no to mention that you also need to call `.anlyze()` for their post-processing.

Here we provide a more efficient way solve this problem,
where the true power of Qurrium as experiment manage toolkit.


### a. Import the instances


In [1]:
from qurry import ShadowUnveil

experiment_shadow = ShadowUnveil()

### b. Preparing quantum circuit

Prepare and add circuits to the `.wave` for later usage.


In [2]:
from qiskit import QuantumCircuit
from qurry.recipe import TrivialParamagnet, GHZ


def make_neel_circuit(n):
    qc = QuantumCircuit(n)
    for i in range(0, n, 2):
        qc.x(i)
    return qc


for i in range(2, 13, 2):
    experiment_shadow.add(TrivialParamagnet(i), f"trivial_paramagnet_{i}")
    experiment_shadow.add(GHZ(i), f"ghz_{i}")
    experiment_shadow.add(make_neel_circuit(i), f"neel_{i}")

experiment_shadow.waves

WaveContainer({
  'trivial_paramagnet_2': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x7931ef5cf470>,
  'ghz_2': <qurry.recipe.simple.cat.GHZ object at 0x7931ef54b950>,
  'neel_2': <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7931ef5eb4d0>,
  'trivial_paramagnet_4': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x7931ef42dc70>,
  'ghz_4': <qurry.recipe.simple.cat.GHZ object at 0x7931ef5ce930>,
  'neel_4': <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7932c7fdb6e0>,
  'trivial_paramagnet_6': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x7931ef42e000>,
  'ghz_6': <qurry.recipe.simple.cat.GHZ object at 0x7931ef42e090>,
  'neel_6': <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7931ef42e300>,
  'trivial_paramagnet_8': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x7931ff70df70>,
  'ghz_8': <qurry.recipe.simple.cat.GHZ object at 0x7931ef763e00>,
  'neel_8': <qiskit.circuit.quantumcircuit.Quantu

### c. Execute multiple experiments at once

Let's demonstrate the true power of Qurrium.


In [3]:
from qurry.qurrent.classical_shadow.arguments import ShadowUnveilMeasureArgs

Preparing a configuration list for multiple experiments with following parameters:

```python
class ShadowUnveilMeasureArgs(total=False):
    """Output arguments for :meth:`output`."""
    shots: int
    """Number of shots."""
    tags: Optional[tuple[str, ...]]
    """The tags to be used for the experiment."""

    wave: Optional[Union[QuantumCircuit, Hashable]]
    """The key or the circuit to execute."""
    times: int
    """The number of random unitary operator. 
    It will denote as `N_U` in the experiment name."""
    measure: Optional[Union[tuple[int, int], int, list[int]]]
    """The measure range."""
    unitary_loc: Optional[Union[tuple[int, int], int, list[int]]]
    """The range of the unitary operator."""
    unitary_loc_not_cover_measure: bool
    """Whether the range of the unitary operator is not cover the measure range."""
    random_unitary_seeds: Optional[dict[int, dict[int, int]]]
    """The seeds for all random unitary operator.
    This argument only takes input as type of `dict[int, dict[int, int]]`.
    The first key is the index for the random unitary operator.
    The second key is the index for the qubit.

    .. code-block:: python
        {
            0: {0: 1234, 1: 5678},
            1: {0: 2345, 1: 6789},
            2: {0: 3456, 1: 7890},
        }

    If you want to generate the seeds for all random unitary operator,
    you can use the function `generate_random_unitary_seeds` 
    in `qurry.qurrium.utils.random_unitary`.

    .. code-block:: python
        from qurry.qurrium.utils.random_unitary import generate_random_unitary_seeds
        random_unitary_seeds = generate_random_unitary_seeds(100, 2)
    """
```


In [4]:
config_list: list[ShadowUnveilMeasureArgs] = [
    {
        "shots": 1024,
        "times": 100,
        "wave": f"{wave_names}_{i}",
        "tags": (wave_names, f"size_{i}"),
    }
    for _ in range(10)
    for i in range(2, 13, 2)
    for wave_names in ["trivial_paramagnet", "ghz", "neel"]
]
print(len(config_list))

180


The `.multiOutput` will return an id of this `multimanager` instance,
which can be used to get the results and post-process them.

Each `multimanager` will export the experiments in a folder you can specify
by setting `save_location` parameter with default location for current directory
where Python executed.
It will create a folder with the name of the `multimanager` instance,
and inside it will create a folder for storing each experiment data.

It will do firstly in the building process, but you can skip it by setting `skip_build_write=True` to save time.
After all experiments are executed, it will export secondly,
which can also be skipped by setting `skip_output_write=True` for no files output.


In [ ]:
multi_exps1 = experiment_shadow.multiOutput(
    config_list,
    summoner_name="qurshady",  # you can name it whatever you want
)
multi_exps1

| MultiManager building...
| Write "qurshady.001", at location "qurshady.001"


| 0/180   0%|          | - MultiManager building... - 00:00 < ?

| MultiOutput running...


| 0/180   0%|          | -  - 00:00 < ?

| Export multimanager...


...

| No quantity to export.
| Export multi.config.json for 7a73b343-ed81-447a-ad4a-f5bc8b3a1782


| 0/179 - Exporting experiments... - 00:00 < ?

| 0/180 - Loading file infomation... - 00:00 < ?

| Exporting file taglist...
| Exporting qurshady.001/qurryinfo.json...
| Exporting qurshady.001/qurryinfo.json done.


'7a73b343-ed81-447a-ad4a-f5bc8b3a1782'

You can check the result of `multiOutput` that we just executed by accessing the `.multimanagers`

In [6]:
experiment_shadow.multimanagers

MultiManagerContainer(num=1, {
  "7a73b343-ed81-447a-ad4a-f5bc8b3a1782":
    <MultiManager(name="qurshady.001", jobstype="local", ..., exps_num=180)>,
})

In [7]:
experiment_shadow.multimanagers[multi_exps1]

<MultiManager(id="7a73b343-ed81-447a-ad4a-f5bc8b3a1782",
  name="qurshady.001",
  tags=(),
  jobstype="local",
  pending_strategy="tags",
  last_events={
    'output.001': '2025-06-17 19:46:21',},
  exps_num=180)>

### d. Run post-processing at once


In [8]:
experiment_shadow.multiAnalysis(
    summoner_id=multi_exps1,
    skip_write=True,
    multiprocess_write=False,
    selected_qubits=[0, 1],
)

| 0/180 - Analysis:  - 00:00 < ?

| "report.001" has been completed.


'7a73b343-ed81-447a-ad4a-f5bc8b3a1782'

In [9]:
print("| Available results:")
for k, v in (
    experiment_shadow.multimanagers[multi_exps1]
    .quantity_container["report.001"]
    .items()
):
    print("| -", k, "with length", len(v))

| Available results:
| - ('trivial_paramagnet', 'size_2') with length 10
| - ('ghz', 'size_2') with length 10
| - ('neel', 'size_2') with length 10
| - ('trivial_paramagnet', 'size_4') with length 10
| - ('ghz', 'size_4') with length 10
| - ('neel', 'size_4') with length 10
| - ('trivial_paramagnet', 'size_6') with length 10
| - ('ghz', 'size_6') with length 10
| - ('neel', 'size_6') with length 10
| - ('trivial_paramagnet', 'size_8') with length 10
| - ('ghz', 'size_8') with length 10
| - ('neel', 'size_8') with length 10
| - ('trivial_paramagnet', 'size_10') with length 10
| - ('ghz', 'size_10') with length 10
| - ('neel', 'size_10') with length 10
| - ('trivial_paramagnet', 'size_12') with length 10
| - ('ghz', 'size_12') with length 10
| - ('neel', 'size_12') with length 10


- Example of the content of `quantity_container`

In [10]:
experiment_shadow.multimanagers[multi_exps1].quantity_container["report.001"][
    ("trivial_paramagnet", "size_10")
][:2]

[{'classical_registers_actually': [1, 0],
  'taking_time': 0.02085137367248535,
  'mean_of_rho': array([[0.25432129+0.j        , 0.34198242-0.00590332j,
          0.22552734+0.00010254j, 0.33767578-0.00153809j],
         [0.34198242+0.00590332j, 0.24822754+0.j        ,
          0.33732422+0.00197754j, 0.22447266+0.00203613j],
         [0.22552734-0.00010254j, 0.33732422-0.00197754j,
          0.24887207+0.j        , 0.33301758+0.00394043j],
         [0.33767578+0.00153809j, 0.22447266-0.00203613j,
          0.33301758-0.00394043j, 0.2485791 +0.j        ]]),
  'purity': np.float64(1.3273232858831232),
  'entropy': np.float64(-0.40851979959784107),
  'estimate_of_given_operators': [],
  'accuracy_prob_comp_delta': nan,
  'num_of_estimators_k': 0,
  'accuracy_predict_epsilon': nan,
  'maximum_shadow_norm': nan,
  'input': {'shots': 1024,
   'num_qubits': 10,
   'selected_qubits': [0, 1],
   'registers_mapping': {0: 0,
    1: 1,
    2: 2,
    3: 3,
    4: 4,
    5: 5,
    6: 6,
    7: 7,


### e. Run post-processing at once with specific analysis arguments

At first, we need to get the each experiment's id in the `multimanager` instance.


In [10]:
expkeys_of_multi_exps1 = list(experiment_shadow.multimanagers[multi_exps1].exps.keys())
print(len(expkeys_of_multi_exps1))

180


1. If you want to run the post-processing for some specific experiments,
   for example, the first 3 experiments we get for the `multimanager` instance.


In [11]:
experiment_shadow.multiAnalysis(
    summoner_id=multi_exps1,
    analysis_name="first_3",
    skip_write=True,
    multiprocess_write=False,
    specific_analysis_args={
        k: (
            {
                "selected_qubits": [0, 1],
            }
            if idx < 3
            else False
        )
        for idx, k in enumerate(expkeys_of_multi_exps1)
    },
)

| 180/180 - Analysis: Skipped 9a0bb3ed-5564-4bfb-9bf2-8a998075dfef in cc9f67e3-fcc4-4399-bf8e-5316e4f90fa0. - 00:00 < 00:00

| "first_3.002" has been completed.


'cc9f67e3-fcc4-4399-bf8e-5316e4f90fa0'

In [12]:
print("| Available results:")
print(
    "| length:",
    sum(
        len(v)
        for v in experiment_shadow.multimanagers[multi_exps1]
        .quantity_container["first_3.002"]
        .values()
    ),
)

| Available results:
| length: 3


2. Or manually specify all the analysis arguments for each experiment.


In [13]:
experiment_shadow.multiAnalysis(
    summoner_id=multi_exps1,
    skip_write=False,
    analysis_name="all_manual",
    multiprocess_write=True,
    specific_analysis_args={
        k: {
            "selected_qubits": [0, 1],  # selected qubits for the analysis
        }
        for idx, k in enumerate(expkeys_of_multi_exps1)
    },
)

| 180/180 - Analysis: | taking time of all rho_m: 0.0164 sec:  - 00:02 < 00:00


| "all_manual.003" has been completed.
| Export multimanager...


| 9/9 - Exporting done - 00:00 < 00:00                           
| 3/3 - exported quantity complete - 00:00 < 00:00    


| Export multi.config.json for cc9f67e3-fcc4-4399-bf8e-5316e4f90fa0


| 179/179 - Exporting experiments... - 00:22 < 00:00
| 180/180 - Loading file infomation... - 00:00 < 00:00

| Exporting file taglist...
| Exporting qurshady.001/qurryinfo.json...
| Exporting qurshady.001/qurryinfo.json done.


'cc9f67e3-fcc4-4399-bf8e-5316e4f90fa0'

In [14]:
print("| Available results:")
print(
    "| length:",
    sum(
        len(v)
        for v in experiment_shadow.multimanagers[multi_exps1]
        .quantity_container["all_manual.003"]
        .values()
    ),
)

| Available results:
| length: 180


### f. Read exported multimanager data


In [ ]:
multi_exps1_reades = experiment_shadow.multiRead(
    save_location=".",
    summoner_name="qurshady.qurry.001",
)

| Retrieve qurshady.001...
| at: qurshady.001


| 0/180   0%|          | - Loading 180 experiments ... - 00:00 < ?

| 180/180 100%|██████████| - Loading 180 experiments ... - 00:06 < 00:00



### Post-Process Availablities and Version Info

In [17]:
from qurry.process import AVAIBILITY_STATESHEET

AVAIBILITY_STATESHEET

 | Qurry version: 0.12.2.dev2
---------------------------------------------------------------------------
 ### Qurry Post-Processing
   - Backend Availability ................... Python Cython Rust   JAX   
 - randomized_measure
   - entangled_entropy.entropy_core_2 ....... Yes    Depr.  Yes    No    
   - entangle_entropy.purity_cell_2 ......... Yes    Depr.  Yes    No    
   - entangled_entropy_v1.entropy_core ...... Yes    Depr.  Yes    No    
   - entangle_entropy_v1.purity_cell ........ Yes    Depr.  Yes    No    
   - wavefunction_overlap.echo_core_2 ....... Yes    Depr.  Yes    No    
   - wavefunction_overlap.echo_cell_2 ....... Yes    Depr.  Yes    No    
   - wavefunction_overlap_v1.echo_core ...... Yes    Depr.  Yes    No    
   - wavefunction_overlap_v1.echo_cell ...... Yes    Depr.  Yes    No    
 - hadamard_test
   - purity_echo_core ....................... Yes    No     Yes    No    
 - magnet_square
   - magnsq_core ............................ Yes    No     No     No  